This notebook refers heavily to these resources: 

1. https://www.youtube.com/watch?v=UU1WVnMk4E8&t=1146s&ab_channel=freeCodeCamp.orgOpen 

2. https://github.com/Infatoshi/fcc-intro-to-llms 

3. https://machinelearningmastery.com/cross-entropy-for-machine-learning/
   
4. https://www.youtube.com/watch?v=IN2XmBhILt4&ab_channel=StatQuestwithJoshStarmer
We are going to use torch library. This allows us to use tensor operation

# Prepare the training data
The training text file

In [1]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Running torch on {}'.format(device))
print(torch.version.cuda)
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

Running torch on cpu
None
﻿The Project Gutenberg eBook of The Wonderful Wizard of Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restri


# 1. Tokenizer

A tokenizer coverts a text into a mathematical object that is easier for the computer to process. There are 3 different level of tokenizers:

1. Word tokenizer: This type of tokenization splits text into individual words. For example, the sentence "Hello, world!" would be tokenized into ["Hello", ",", "world", "!"]. Word tokenization is commonly used in many NLP tasks.

2. Subword Tokenizer: Subword tokenization breaks down words into smaller units called subwords. This approach is particularly useful for handling languages with complex morphology or dealing with out-of-vocabulary words. Examples of subword tokenization algorithms include Byte Pair Encoding (BPE) and WordPiece.

3. Character Tokenizer: Character tokenization represents each character in the text as a separate token. This approach is useful when dealing with languages that do not have clear word boundaries or for tasks like text generation.


Here, we will use the simplest tokenizer, the character tokenizer. We can build our own tokenizer through the code written below:

In [10]:
chars = sorted(set(text))
print(chars)
vocubary_size = len(chars)
print("The amount of unique character is {}".format(vocubary_size))
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l ])

print('Encoding ...: {}'.format(encode('hello')))
print('Decoding ... {}'.format(decode(encode('hello'))))


['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
The amount of unique character is 89
Encoding ...: [62, 59, 66, 66, 69]
Decoding ... hello


Once the encoding function is defined, we shall encode our text. Since the text contains a lot of characters, after the encoding our list will be super large, which will not be efficient to operate on. Instead, we are going to store all the encoded text data into a torch.tensor data structure. Storing the data into tensor will allow us to perform tensor operation on it. 

In [3]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([88, 46, 62, 59,  1, 42, 72, 69, 64, 59, 57, 74,  1, 33, 75, 74, 59, 68,
        56, 59, 72, 61,  1, 59, 28, 69, 69, 65,  1, 69, 60,  1, 46, 62, 59,  1,
        49, 69, 68, 58, 59, 72, 60, 75, 66,  1, 49, 63, 80, 55, 72, 58,  1, 69,
        60,  1, 41, 80,  0,  1,  1,  1,  1,  0, 46, 62, 63, 73,  1, 59, 56, 69,
        69, 65,  1, 63, 73,  1, 60, 69, 72,  1, 74, 62, 59,  1, 75, 73, 59,  1,
        69, 60,  1, 55, 68, 79, 69, 68, 59,  1])


# 2. Preparing the data for the network training

We are going to split the text file into training data and validation data. This is normally done through a 8:2 split. The splitting is important to make the model generative instead of spliting out the whole text that it has been fed. 

Feeding the whole chunck of text data to the computer and letting it iterate one by one can be very time consuming. To save time, we can divide the training data into batches and let the computer processes these batches of data at the same time. This is known as parallelism, and will help us to save up a lot of computational time. 

In language model, a block size refers to the length of the sequences or blocks of text used during training. Sequences longer than the block size will be divided into smaller blocks or truncated. Therefore, block size determines the maximum length of sequences that the model can handle during training. 

We will define a function that randomly select a block of text from the training/validation data. The output of this function will consist of the random block of text selected from the data stored, as well as a block of text that follows immediately from the selected data, both output stored as torch.tensor. 

In [5]:
block_size = 8
batch_size = 4
n = int(0.8*len(data)) # spliting the data 
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    '''
    Randomly obtain a chunck of input data from either the train set or validation set. 
    Store the next sequence to a tensor (y).
    Output the input and output data 
    '''
    data = train_data if split == 'train' else val_data
    print(len(data) - block_size)
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    x = torch.stack([data[i:i+block_size] for i in ix]) 
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


181672
inputs:
tensor([[ 1, 56, 63, 61,  1, 55, 68, 58],
        [69, 58, 67, 55, 68,  1, 55, 68],
        [79, 63, 68, 61,  1, 74, 62, 59],
        [68,  1, 55, 73,  1, 70, 69, 73]])
targets:
tensor([[56, 63, 61,  1, 55, 68, 58,  1],
        [58, 67, 55, 68,  1, 55, 68, 58],
        [63, 68, 61,  1, 74, 62, 59,  1],
        [ 1, 55, 73,  1, 70, 69, 73, 73]])


# The gist of neural network
I highly recommend getting the main idea of the neural network by watching the video from Statquest with Josh Stammer. To sum it up, a neural network performs a linear regression-like operation on the input vector (which is the encoded data). Each node performs operations that are of similar form:
$$ \vec{y}_i = F(W\vec{x} + \vec{b})_i = (W\vec{x} + \vec{b})_i \text{ if }  (W\vec{x} + \vec{b})_i \geq 0; 0 \text{ if } (W\vec{x} + \vec{b})_i 
< 0 $$

The embedded function within $F(x)$ is performing linear regression to the input data $\vec{x}$. The parameters $w_i\in W$ and $b_i \in b$ are unknown and the purpose of training is to obtain the best value for these parameters. For starter, $W$ and $\vec{b}$ can be first randomly assigned, and improved from there on. Once we have calculated the $W\vec{x} + \vec{b}$, it will pass through a function $F$ that judges the `` degree of correctness". This function is known as the activation function. Once a score has been assigned to the output, it will be compared to the true value. If the true value is known, this is supervised learning, and the goal is to minimize the error between the true and predicted value. If the true value is unknown, this is unsupervised learning, and often a true value will be inferred, and we will still try to minimize the error. 

What we have described so far is the forward propagation process, $\vec{y} = F(W\vec{x} + \vec{b})$. To improve the weight matrix $W$, we will perform a backward propagation process. This is the process where we try to perform gradient descent on matrix $W$ to obtain the best value.

# Writing the model

Now that the training data is in place, we are going to write the training model. Before we get into it, lets first define a few mathematical objects.

2. Activation function 

An activation function assigns weight to each input data after passing through a layer. A common example includes linear activation, ReLU activation etc. 

3. Loss function: A loss function is a mathematical function that quantifies the difference between predicted and actual values in a machine learning model $Loss = \|Predicted - Actual\|$. It measures the model's performance and guides the optimization process by providing feedback on how well it fits the data. The closer the predicted value is from the actual data, the smaller the loss.

A loss function can be any function (for example, a linear function $loss(x) = 1 - log_2(p(x))$, as long as it rewards data that provides prediction that is closer to the actual data, and punishes data that provides prediction that is far away from the data
 
    a. Entropy: Entropy is a measure of the amount of information stored in a data point $x$. Given that the probability of finding data point $x$ as $p(x)$, the entropy carried by data point $x$, $h(x)$ is given by $h(x) = -log_2(P(x))$. In the context of machine learning, entropy tells how much information is stored in a set of data. For example, if data contains a lot of the letter $w$, the next letter $w$ will give us less information, since we have already expected it to exist due to its large probability. However, if the next letter is $x$, which appears less in the text, this letter $x$ carries more information because it seldom appears in our data. This gives us a chance to learn about the underlying pattern instead of repeating $w$. Therefore we assign $x$ with a larger entropy than $w$.

    If we have a total entropy $H = \sum_x h(x)$ that is very low, it suggests a very skewed data set, since there is not much valuable information generated from the dataset. On the other hand, a high total entropy suggests a balanced data set, and is probably the one more valuablee for the network training.  

    b.  Cross-entropy: Cross entropy of x, written as $h_x(y) = ln(p(y))$ is a measure of the difference between two random variable $x$ and $y$.  We use cross entropy because it has the following nice properties:
       a. when the probability $p(y)$ is very high (close to 1), cross entropy returns a very low value. When p(y)= 1, the cross entropy is 0.
       b. when the probability $p(y)$ is very low (close to zero), cross entropy returns a very large value.
       c. cross entropy has a well-defined derivative at the range (0,1].

    Now, imagine that $p(y)$ returns the probability of getting the right prediction by the model. These properties make cross entropy a very good loss function. We want a function that greatly rewards the training model when the prediction generated from the model is close to the actual value, and greatly punish data that doesn't offer a lot of new information.  
 
Note that in most literature, the quantity $h(x) = -log(P(x))$ is called information, $H(x) = \sum_x h(x)$ is called entropy, and $H(X,Y) = \sum_{x\in X, y in Y} h(x,y)$ is known as cross-entropy. I call $h(x)$ and $h(x,y)$ as total entropy and total cross entropy because of the summation operation, and also I do not find it helpful to give the respective quantity $h(x)$ and $h(x,y)$ a new term, since we already have a lot of terms floating around. 

Therefore, the simplest neural network model can be summarised by the following figure:
 insert figure here

 There are more complex models out there, some include a filter before performing the linear operation on the data (this is known as a convolutional neural network). Building your neural network from scratch allows you to have greater freedom to design the network architecture that best fits the problem you have.

Let's try to build our language model using the simplest framework we described above: any form of linear operator, an activation function, and then a loss function to improve the model. For this example, we will be using a special kind of operator known as the embedding operator. An embedding operator creates a 

The vocabulary size must be greater or equal to the amount of unique tokens you have in the input data. Below is the pseudo-code for how an embedding layer looks like. 

In [22]:
import torch

# Initialization
vocab_size = 100
embedding_dim = 5
embedding_table = torch.randn(vocab_size, embedding_dim)

#print(embedding_table)
# Lookup Operation
string1 = encode("Hello")
string2 = encode("world")
print(string1, string2)
input_tensor = torch.tensor([string1, string2])  # Example input tensor
batch_size, sequence_length = input_tensor.size()
embedding_output = torch.zeros(batch_size, sequence_length, embedding_dim)

for i in range(batch_size):
    for j in range(sequence_length):
        token_index = input_tensor[i, j]
        embedding_vector = embedding_table[token_index]
        embedding_output[i, j] = embedding_vector

print(embedding_output)

[34, 59, 66, 66, 69] [77, 69, 72, 66, 58]
tensor([[[-0.4090, -1.0636,  0.7896, -2.2421, -0.8987],
         [ 0.6725,  0.3870, -0.1524,  0.3461,  1.3665],
         [-0.0065,  2.1077,  2.7225, -0.1317, -0.7571],
         [-0.0065,  2.1077,  2.7225, -0.1317, -0.7571],
         [-0.4818,  1.0238,  0.2085, -1.3190, -0.2677]],

        [[ 0.7989,  1.7100,  0.9505,  1.6261, -0.5539],
         [-0.4818,  1.0238,  0.2085, -1.3190, -0.2677],
         [ 0.5312,  0.2973,  1.4177,  1.3318,  2.0641],
         [-0.0065,  2.1077,  2.7225, -0.1317, -0.7571],
         [ 1.0608,  0.3211,  1.0668, -0.3635,  2.1404]]])


In [15]:

n_embd = 384

class languagemodel(nn.Module):
    def __init__(self, vocab_size,  n_embd):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        self.net = nn.Sequential(
                    nn.Linear(n_embd, 4 * n_embd),
                    nn.ReLU(),
                    nn.Linear(4 * n_embd, n_embd),
                    nn.Dropout(dropout),
            
    def forward(self, x):
        return self.net(x)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) # perform embedding to the word and store it as logits

        logits = self.linear()
        return logits



SyntaxError: '(' was never closed (560203714.py, line 7)